In [118]:
# Import numpy and pandas packages 
import numpy as np
import pandas as pd  
import math as ma
    


#####Create a new data frame with only the required dependent and independent variables 
#####Ignore other columns in the data frame
#arg1 is the data frame 
## Example of arg2 is ['y', ['x1', 'x2', 'x3']] or ['Price',['Mileage']]
def cleanup_dataFrame (arg1,arg2):

    ###Create a list of variable names - both dependendent and independent varibles only
    data1_var_new = []
    data1_var_new.append(arg2[0])
    #print(data1_var_new)
    for word in arg2[1]:
        #print(word)
        data1_var_new.append(word)

    print(data1_var_new)

    ###Create a new data frame with only the dependent and independent variables
    data1_new_frame = arg1.loc[:, data1_var_new]
    #print(data1_new_frame)
    return data1_new_frame





#Average of any variable
#arg1 is the data frame 
## Example of arg2 is ['y', ['x1', 'x2', 'x3']] or ['Price',['Mileage']]
def avg_variable (arg1,arg2):
    
    average = {arg2[0]: round(arg1[arg2[0]].mean(),2)}
    #print(average)
    i=0
    for ind_var in arg2[1]:
        average.update({arg2[1][i]: round(arg1[arg2[1][i]].mean(),2)})
        i=i+1
    #print(average)
    return average

#number of entries/ sample space
def find_threshold (arg1):
    return round(ma.sqrt(1.96/(arg1-1)),2)




#arg1 is the dataframe and arg2 is the list of dependent and independent variables
# Example of arg2 is ['y', ['x1', 'x2', 'x3']] or ['Price',['Mileage']]
def modelLinearRegression (arg1, arg2):
 
    num_indep_var = len(arg2[1])
    print("Number of independent Variables: ",num_indep_var)
    print("The list of variables: ",arg2)
    #print(arg1)
    
    #cleanup the dataframe - remove unncessary columns
    arg1 = cleanup_dataFrame (arg1,arg2)
    

    ###Find Threshold
    sample_space = len(arg1)
    #print(sample_space)
    threshold = find_threshold(sample_space)
    
    #print("Threshold:",threshold)
    
    

    ###Find Average of all variables
    average = avg_variable (arg1,arg2)
    #print("Average:", average)

    
    ###Create new columns difference between avg and current value
    col_name = arg2[0]+str('-avg')
    #print(average[arg2[0]])
    arg1[col_name] =  (arg1[arg2[0]] - average[arg2[0]] )
    #print(arg1[col_name].sum()) ###To test if sum is zero.
    #print(arg1)
    
    i=0
    for ind_var in arg2[1]:
        col_name = ind_var+str('-avg')
        arg1[col_name] =  (arg1[arg2[1][i]] - average[arg2[1][i]] )
        #print(arg1[col_name].sum()) ###To test if sum is zero.
        i=i+1
    
    #print(arg1)
    
    
    ###Create new columns for square for (y-avg) 
    col_name = arg2[0]+str('-avg')
    new_col_name = str('sq-')+arg2[0]+str('-avg')
    #print(new_col_name)
    arg1[new_col_name] =  pow(arg1[col_name],2)
    
    #print(arg1)
    ###Create new columns for square for (x-avg) 
    i=0
    for ind_var in arg2[1]:
        col_name = ind_var+str('-avg')
        new_col_name = str('sq-')+ind_var+str('-avg')
        arg1[new_col_name] =  pow(arg1[col_name],2)
        
        i=i+1
    
    #print(arg1)
    
    
    ###Create new columns for dependent variable and independent variable (x-avg)(y-avg)
    dep_col_name = arg2[0]+str('-avg')
    #print("Dependent Column name:",dep_col_name)
    i=0
    for ind_var in arg2[1]:
        col_name = ind_var+str('-avg')
        new_col_name = '('+dep_col_name+')('+ind_var+str('-avg')+')'
        #print("Independent Column name:",col_name)
        #print("New Column name:",new_col_name)
        arg1[new_col_name] =  arg1[col_name]*arg1[dep_col_name]
        
        i=i+1
    
    #print(arg1)
    
    
    ####Calculating the correlation coefficients
    correlation_coefficients = {}
    dep_col_name = arg2[0]+str('-avg')
    sq_dep_col_name = str('sq-')+arg2[0]+str('-avg')
    i=0
    for ind_var in arg2[1]:
        sq_in_col_name = str('sq-')+ind_var+str('-avg')
        in_de_col_name = '('+dep_col_name+')('+ind_var+str('-avg')+')'
        corr_coef = round(arg1[in_de_col_name].sum()/ma.sqrt(arg1[sq_in_col_name].sum()*arg1[sq_dep_col_name].sum()),2)
        correlation_coefficients[ind_var] = corr_coef
        i=i+1
    
    print("Correlation Coefficients are: ",correlation_coefficients)
    print("Threshold: ",threshold)
    #print("arg2 as passed in the function: ", arg2)
    
    ####Find and list which independent variables are significant
    signific_ind_vars = []
    for ind_var in arg2[1]:
        if (abs(correlation_coefficients[ind_var]) > threshold):
            signific_ind_vars.append(ind_var)
    print(":::Significant Independent Variables are:::",signific_ind_vars)
    
    
    
    ###Create new columns for all independent significant variables for ex Y1*X1
    rhs_matrix_col_names = []
    lhs_matrix_col_names = []
    
    
    for ind_var in signific_ind_vars:
        dep_indep_col_name = arg2[0]+str('*')+ind_var
        arg1[dep_indep_col_name] =  arg1[arg2[0]]*arg1[ind_var]
        ###store column names of rhs matrix
        rhs_matrix_col_names.append(dep_indep_col_name)
        #rhs_matrix_col_names.append(rhs_matrix_col_names_row)
    rhs_matrix_col_names.append(arg2[0])
        
    
    for ind_var in signific_ind_vars: 
        lhs_matrix_col_names_row = []
        for ind_var1 in signific_ind_vars:
            indep_indep_col_name = ind_var1+str('*')+ind_var
            arg1[indep_indep_col_name] =  arg1[ind_var1]*arg1[ind_var]
            ###store column names of lhs matrix
            lhs_matrix_col_names_row.append(indep_indep_col_name)
    
        lhs_matrix_col_names_row.append(ind_var)
        lhs_matrix_col_names.append(lhs_matrix_col_names_row)
    
    
    #Adding last equation, 
    #Last variable is total sample space (N)
    lhs_matrix_col_names_row = []
    str1 = 'SS'
    for ind_var in signific_ind_vars: 
        lhs_matrix_col_names_row.append(ind_var)
    lhs_matrix_col_names_row.append(str1)
    lhs_matrix_col_names.append(lhs_matrix_col_names_row)
        
    #print(arg1)
    
    #print ("RHS Column names:", rhs_matrix_col_names)
    #print ("LHS Column names:", lhs_matrix_col_names)
    
    
    rhs_matrix = []
    lhs_matrix = []
    rhs_matrix_row = []
   

    i=0
    for item in rhs_matrix_col_names:
        #rhs_matrix_row.append(round(arg1[rhs_matrix_col_names[i]].sum(),2))
        #print("HERe is the data:",rhs_matrix_col_names[i])
        #rhs_matrix.append(rhs_matrix_row)
        rhs_matrix.append([round(arg1[rhs_matrix_col_names[i]].sum(),2)])
        i=i+1
        
    #print ("RHS Matrix:", rhs_matrix)
    
    

    for item in lhs_matrix_col_names:
        lhs_matrix_row = []
        #print("HERE IS ITEM::: item",item )
        for item1 in item:
            #print("HERE IS ITEM1::: item1",item1 )
            if(item1 == 'SS'):
                lhs_matrix_row.append(round(arg1[arg2[0]].count(),2))
            #lhs_matrix_row.append(round(arg1[item1].sum(),2))
            #lhs_matrix.append(lhs_matrix_row)
            else:
                lhs_matrix_row.append(round(arg1[item1].sum(),2))
            
        lhs_matrix.append(lhs_matrix_row)
        
    #print ("LHS Matrix:", lhs_matrix)
    
    
    
    
    
    ###NUMPY MAtrix creation
    
    RHS = np.array(rhs_matrix)
    LHS = np.array(lhs_matrix)
    #print("Matrix LHS is:\n",LHS)
    #print("Matrix RHS is:\n",RHS)
    
    #SOLVE THE EUATIONS using matrix inverse and multiplication
    LHS_inv = np.linalg.inv(LHS)
    #print("Inverse of Matrix LHS is:\n",LHS_inv)

    
    SOL = np.matmul(LHS_inv,RHS)
    #print("Solution of the equations are :\n",SOL)
    sol_list = SOL.tolist()
    print("Solution of the equations are :\n",sol_list)
    
    
    ###Creating the Linear Regression Model
    linRegModel = []
    linRegModel = arg2[0]+str('=')
    i=0
    for ind_var in signific_ind_vars:
        if(i != 0):
            linRegModel = linRegModel + str('+')
        linRegModel = linRegModel + str(round(sol_list[i][0],2)) + ind_var
        i=i+1
    
    linRegModel = linRegModel + str('+') + str(round(sol_list[-1][0],2))
    print("\nThe LINEAR REGRESSION MODEL is:", linRegModel)
    #####
    
    
    return linRegModel









# making a data frame from the File
#Can be done by Team 1 or Team2 
data1 = pd.read_csv("Automobiles_price_mileage_country.csv") 
data1_var = ['Price',['Mileage']]

#####
data2 = pd.read_csv("multiRegressionData.csv")  
# list(data)
#list(data1.columns) 
#print(data1)

#list(data2.columns) 
#print(data2)

data2_var = ['Y',['X1','X2']]
#print(type(data2_var))
#print(data2_var)
#print(data2_var[0])
#print(len(data2_var[1]))
#print(data2_var[1][0])
#print(data2_var[1][2])


print("\n\n")

modelLinearRegression(data1,data1_var)
print("\n\n")

modelLinearRegression(data2,data2_var)
print("\n\n")






Number of independent Variables:  1
The list of variables:  ['Price', ['Mileage']]
['Price', 'Mileage']
Correlation Coefficients are:  {'Mileage': -0.63}
Threshold:  0.21
:::Significant Independent Variables are::: ['Mileage']
Solution of the equations are :
 [[-539.0493151923329], [25912.747095782717]]

The LINEAR REGRESSION MODEL is: Price=-539.05Mileage+25912.75



Number of independent Variables:  2
The list of variables:  ['Y', ['X1', 'X2']]
['Y', 'X1', 'X2']
Correlation Coefficients are:  {'X1': 0.82, 'X2': 0.77}
Threshold:  0.42
:::Significant Independent Variables are::: ['X1', 'X2']
Solution of the equations are :
 [[0.8546099290780376], [1.5063323201621586], [3.6512158054720203]]

The LINEAR REGRESSION MODEL is: Y=0.85X1+1.51X2+3.65



